# HW4

## Part 1: Automatic Differentiation [30 pts]

### Problem 1: Basics

#### Going to do some AD on the function
$$
\begin{align*}
    f(x,y,z) = \frac{1}{xyz} 
    + \sin\left(\frac{1}{x} + \frac{1}{y} + \frac{1}{z}\right)\quad.
\end{align*}
$$

#### Parts 1 and 2) Evaluation of $f(a)$ and $\nabla f(a)$ for $a = (1, 2, 3)$

Trace    | Elementary function | Current Value                               | Elementary Function Derivative                    |$\nabla_x Value$                                |$\nabla_y$ Value                             |$\nabla_z$ Value         
---------|---------------------|---------------------------------------------|---------------------------------------------------|------------------------------------------------|---------------------------------------------|-----------------
$x_1   $ | $x_1=x$             | $1$                                         | $\dot x_1$                                        | $1$                                            | $0$                                         | $0$                
$x_2   $ | $x_2=y$             | $2$                                         | $\dot x_2$                                        | $0$                                            | $1$                                         | $0$                
$x_3   $ | $x_3=z$             | $3$                                         | $\dot x_3$                                        | $0$                                            | $0$                                         | $1$                
$x_4   $ | $x_1x_2x_3$         | $6$                                         | $x_1(x_2\dot x_3 + \dot x_2x_3) + \dot x_1x_2x_3$ | $6$                                            | $3$                                         | $2$
$x_5   $ | $\frac{1}{x_4}$     | $1/6$                                       | $-\frac{\dot x_4}{x_4^2}$                         | $-\frac{1}{6}$                                 | $-\frac{1}{12}$                             | $-\frac{1}{18}$
$x_6   $ | $\frac{1}{x_1}$     | $1$                                         | $-\frac{\dot x_1}{x_1^2}$                         | $-1$                                           | $0$                                         | $0$ 
$x_7   $ | $\frac{1}{x_2}$     | $1/2$                                       | $-\frac{\dot x_2}{x_2^2}$                         | $0$                                            | $-\frac{1}{4}$                              | $0$                                     
$x_8   $ | $\frac{1}{x_3}$     | $1/3$                                       | $-\frac{\dot x_3}{x_3^2}$                         | $0$                                            | $0$                                         | $-\frac{1}{9}$         
$x_9   $ | $x_6+x_7+x_8$       | $11/6$                                      | $\dot x_6 + \dot x_7 + \dot x_8$                  | $-1$                                           | $-\frac{1}{4}$                              | $-\frac{1}{9}$
$x_{10}$ | $\sin(x_{9})$       | $\sin\left(\frac{1}{6}\right)$              | $\dot x_9\cos(x_9)$                               | $-\cos\left(\frac{11}{6}\right)$               | $-\frac{1}{4}\cos\left(\frac{11}{6}\right)$ | $-\frac{1}{9}\cos\left(\frac{11}{6}\right)$           
$x_{11}$ | $x_5 + x_{10}$      | $\frac{1}{6}+\sin\left(\frac{11}{6}\right)$ | $\dot x_5 + \dot x_{10}$                          | $-\frac{1}{6} - \cos\left(\frac{11}{6}\right)$ | $-\frac{1}{12} - \frac{1}{4}\cos\left(\frac{11}{6}\right)$ | $-\frac{1}{18} - \frac{1}{9}\cos\left(\frac{11}{6}\right)$

### Problem 2: AD with dual numbers

Re-writing in terms of dual numbers, the $1/x$ term can be expressed as
$$
\begin{align*}
    \frac{1}{x+\epsilon x'} 
    = \frac{x - \epsilon x'}{(x + \epsilon x')(x - \epsilon x')}
    = \frac{x - \epsilon x'}{x^2} = \frac{1}{x} - \epsilon\frac{x'}{x^2}\quad.
\end{align*}
$$

Subbing this into the equation for $f(x + \epsilon x')$ and using the 
$\sin(\alpha + \epsilon\beta)$ identity,
$$
\begin{align*}
    f(x + \epsilon x') &= \frac{1}{x} - \epsilon\frac{x'}{x^2} 
    + \sin\left(\frac{1}{x} - \epsilon\frac{x'}{x^2}\right)
    = \frac{1}{x} - \epsilon\frac{x'}{x^2} + \sin\frac{1}{x}
    - \epsilon\frac{x'}{x^2}\cos\frac{1}{x} \\
    &= \frac{1}{x} + \sin\frac{1}{x} 
    + \epsilon\left[-\frac{x'}{x^2}\left(1 + \cos\frac{1}{x}\right)\right]\quad.
\end{align*}
$$

The dual part is our derivative,
$$
\begin{align*}
    \boxed{f'(x) = - \frac{1}{x^2}\left(1 + \cos\frac{1}{x}\right)}\quad, 
\end{align*}
$$

where $x' = \text dx/\text dx = 1$.

### Problem 3: Toy forward mode problem

We want to perform AD on the function $f(x) = \alpha x + 3$, so the addition and multiplication operators will need to be overloaded. The rules for addition remain the same (plus `__radd__` to maintain commutivity) but multiplication will follow the new rule:
$$
\begin{align*}
    a\times b = (a_r + \epsilon a_d)(b_r + \epsilon b_d)
    = a_rb_r + \epsilon(a_rb_d + a_db_r)\quad,
\end{align*}
$$

where $x_r$, $x_d$ is the real/dual part of dual number $x$, respectively, and $\epsilon^2\equiv 0$.

Will also implement `__rmult__` to maintain multipication commutivity, and `try` and `except` blocks to handle possible `AttributeErrors` when calling our `AutoDiffToy` class.

In [124]:
class AutoDiffToy():
    
    def __init__(self, a):
        # assumes f(x) = alpha*x + 3
        self.val = a  # will return f(a)
        self.der = 0 # will return f'(a)
        
    def __add__(self, other):
        # assumes f AutoDiffToy object already created
        try:
            self.val += other.val
        except AttributeError:
            self.val += other
        return self
    __radd__ = __add__
    
    def __mul__(self, other):
        f = AutoDiffToy(self.val)
        try:
            f.val = self.val*other.val
            f.der = other.val
        except AttributeError:
            f.val = self.val*other
            f.der = other
        return f
    __rmul__ = __mul__

# configure object
a = 2.0
x = AutoDiffToy(a)

alpha = 2.0

# test runs, output should be: 7.0 2.0
f_test = [alpha * x + 3.0,
         x * alpha + 3.0,
         3.0 + alpha * x,
         3.0 + x * alpha]

for i, test in enumerate(f_test):
    f = test
    print(f'Test {i}: {f.val} {f.der}')

Test 0: 7.0 2.0
Test 1: 7.0 2.0
Test 2: 7.0 2.0
Test 3: 7.0 2.0


With these tests at least, it looks like the implementation of `AutoDiffToy` passes.